In [2]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm

from dateutil.parser import parse as DateParser
import os

def _dict_lower_keys(d):
    return {k.lower():v for k,v in d.items()}


condir = '/home/lordluen/Code/InsightDS/Legis_data/contributions/'

In [3]:
FIELD_SPECS={
    'year': int,
    'amount': float,
    'registrantid': int,
    'received': DateParser,
    'contributiondate': DateParser,
    'contributiontype': str,
    'honoree': str,
}

In [8]:
path_to_file = condir+"2016_2_6_1_LD203.xml"
print(path_to_file)

/home/lordluen/Code/InsightDS/Legis_data/contributions/2016_2_6_1_LD203.xml


In [77]:
bar = {key for key in FIELD_SPECS.items()}
print(bar)
for key,cast in FIELD_SPECS.items():
    bar[key].append(cast(data[key])) 

{('honoree', <class 'str'>), ('amount', <class 'float'>), ('received', <function parse at 0x7f56b415c7b8>), ('contributiondate', <function parse at 0x7f56b415c7b8>), ('registrantid', <class 'int'>), ('year', <class 'int'>), ('contributiontype', <class 'str'>)}


TypeError: 'set' object is not subscriptable

In [9]:
fd = open(path_to_file)
dom = ET.fromstring(fd.read())

In [10]:
counter = 0
data = {}
data_topush = {}
foo = True
alldat = pd.DataFrame()
for filing in tqdm(dom.findall('.//Filing')):
    for reg in filing.findall('.//Registrant'):
        for child in filing.findall('.//Contribution'):
            data.clear()
            data_topush.clear()
            data.update(_dict_lower_keys(filing.attrib))
            data.update(_dict_lower_keys(child.attrib))
            data.update(_dict_lower_keys(reg.attrib))
            data_topush = {key:cast(data[key]) for key,cast in FIELD_SPECS.items()}

            for key,cast in FIELD_SPECS.items():
                alldat.loc[counter,key] = cast(data[key])
            counter = counter + 1
            
alldat['year'] = alldat['year'].astype(int)
#alldat['registrantid'] = alldat['registrantid'].astype(int)


100%|██████████| 10/10 [00:00<00:00, 39162.50it/s]


In [11]:
alldat.head(20)

""


In [84]:
print(type(alldat['year'].iloc[0]))

<class 'numpy.float64'>


In [87]:
alldat['year'] = alldat['year'].astype(int)

In [12]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder

from operator import attrgetter

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [14]:
raw_bills = bills.read_bills( start_year = 2015 )

In [15]:
type(raw_bills)

generator

In [16]:
data = pd.DataFrame.from_dict( raw_bills )

100%|██████████| 119/119 [00:01<00:00, 88.91it/s]


In [17]:
data.head(2)

,category,chamber,congress,date,number,result,subjects,top_subject,type
0,amendment,s,114,2015-03-26 23:23:00-04:00,11,Amendment Agreed to,"[Accounting and auditing, Administrative law a...",Economics and public finance,sconres
1,amendment,s,114,2015-12-03 19:06:00-05:00,3762,Amendment Rejected,"[Abortion, Appropriations, Drug trafficking an...",Health,hr


In [18]:
foo = list()
for i in data['subjects']:
    foo = foo+i

In [19]:
print(foo[0])

Accounting and auditing


In [20]:
nfoo = np.asarray(foo)

In [21]:
bar = np.unique(nfoo)

In [22]:
print(bar)

['ASEAN countries' 'Abortion' 'Academic performance and assessments'
 'Access Board' 'Accidents' 'Accounting and auditing'
 'Administrative Conference of the U.S.'
 'Administrative law and regulatory procedures' 'Administrative remedies'
 'Adoption and foster care' 'Adult education and literacy'
 'Advanced technology and technological innovations' 'Advisory bodies'
 'Afghanistan' 'Africa' 'African Development Foundation'
 'Age discrimination' 'Aging' 'Agricultural conservation and pollution'
 'Agricultural education' 'Agricultural insurance'
 'Agricultural marketing and promotion'
 'Agricultural practices and innovations'
 'Agricultural prices, subsidies, credit' 'Agricultural research'
 'Agricultural trade' 'Agriculture and food' 'Air quality' 'Alabama'
 'Alaska' 'Alaska Natives and Hawaiians' 'Alcoholic beverages' 'Alliances'
 'Allied health services' 'Alternative and renewable resources'
 'Alternative dispute resolution, mediation, arbitration'
 'Alternative treatments' 'American Ba

In [23]:
print(bar[0])

ASEAN countries


In [25]:
print(bar[-7])

Women's rights


In [29]:
subs = pd.DataFrame()

In [35]:
subs['subjects'] = bar
subs.head(-1)

,subjects
0,ASEAN countries
1,Abortion
2,Academic performance and assessments
3,Access Board
4,Accidents
5,Accounting and auditing
6,Administrative Conference of the U.S.
7,Administrative law and regulatory procedures
8,Administrative remedies
9,Adoption and foster care


In [36]:
#set up engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)



In [38]:
sqlCommands.append_to_database(dbname,'topics',subs,engine)

In [ ]:
query = "SELECT * FROM topics;"

bills_df = pd.read_sql_query(query,engine)

In [120]:
import os
import numpy as np
import pandas as pd
import ujson as json
from tqdm import tqdm
from sqlCommands import append_to_database

from dateutil.parser import parse as DateParser

In [121]:
bill_file = '/home/lordluen/Code/InsightDS/Legis_data/votes/108/bills/s/s1870/data.json'
data = json.load(open(bill_file))

In [122]:
subjects=list(data['subjects'])
top_subject=data['subjects_top_term']
bill_type = data['bill_type']
status = data['status']
tsponsor = data['sponsor']
intro_date = DateParser(data['introduced_at'])
try:
    final_date = DateParser((data['summary'])['date'])
except:
    final_date = DateParser('9999-09-09')
if tsponsor['type'] == 'person': #lets only use bills from people, not committees. Since funding would be hard to trace otherwise.
    sponsor = [tsponsor['title'],tsponsor['name'],tsponsor['state']]
tcosponsors = data['cosponsors']
cosponsors = list()
if len(tcosponsors) > 0:
    for co in tcosponsors:
        cosponsors.append([co['title'],co['name'],co['state']])
foo = pd.DataFrame()
foo['subjects'] = [subjects]
foo['top_subject'] = top_subject
foo['bill_type'] = bill_type
foo['status'] = status
foo['sponsor'] = [sponsor]
foo['cosponsors'] = [cosponsors]
foo['intro_date'] = intro_date
foo['final_date'] = final_date

In [123]:
foo

,subjects,top_subject,bill_type,status,sponsor,cosponsors,intro_date,final_date
0,"[Labor and employment, Unemployment insurance]",Labor and employment,s,REFERRED,"[Sen, Smith, Gordon H., OR]","[[Sen, Wyden, Ron, OR]]",2003-11-17,2003-11-17


In [95]:
sponsor = info['sponsor']
subjects = info['subjects']
cosponsors = info['cosponsors']

In [96]:
print(type(sponsor))
print(type(subjects))
print(type(cosponsors))

<class 'list'>
<class 'list'>
<class 'list'>


In [98]:
print(sponsor)
print(subjects)
print(cosponsors)

['Rep', 'Weygand, Robert A.', 'RI']
['Chemicals', 'Commerce', 'Dyes and dyeing', 'Environmental protection', 'Foreign trade and international finance', 'Tariff']
[]


In [104]:
bar = pd.DataFrame()
bar['sponsor'] = [sponsor]
bar['subjects'] = [subjects]
bar['cosponsors'] = [cosponsors]
bar

,sponsor,subjects,cosponsors
0,"[Rep, Weygand, Robert A., RI]","[Chemicals, Commerce, Dyes and dyeing, Environ...",[]


In [94]:
foo = pd.DataFrame.from_dict(info)
foo

ValueError: arrays must all be same length

In [89]:
print(info['sponsor'])

['Rep', 'Weygand, Robert A.', 'RI']


In [82]:
foo = cosponsors[0]

IndexError: list index out of range

In [73]:
print(foo)

{'district': None, 'withdrawn_at': None, 'state': 'OR', 'sponsored_at': '2003-11-17', 'title': 'Sen', 'thomas_id': '01247', 'name': 'Wyden, Ron'}


In [74]:
print(type(foo))

<class 'dict'>


In [75]:
print(info['cosponsors'])

[[['Sen', 'Wyden, Ron', 'OR']]]


In [83]:
print(info)

{'sponsor': ['Rep', 'Weygand, Robert A.', 'RI'], 'cosponsors': [], 'status': 'REFERRED', 'top_subject': 'Foreign trade and international finance', 'type': 'hr', 'subjects': ['Chemicals', 'Commerce', 'Dyes and dyeing', 'Environmental protection', 'Foreign trade and international finance', 'Tariff']}


In [84]:
#bleh = ['a','b','c']
bleh = info['sponsor']
print(bleh)
print(type(bleh))
print(bleh[0])
print(bleh[1])
print(bleh[2])
bar = pd.DataFrame(columns=['sponsor'])
bar['sponsor'] = [bleh]
#bar['sponsor'].loc[0] = list()
#bar['sponsor'].iloc[0] = bar['sponsor'].iloc[0].append(bleh)
bar.head(2)

['Rep', 'Weygand, Robert A.', 'RI']
<class 'list'>
Rep
Weygand, Robert A.
RI


,sponsor
0,"[Rep, Weygand, Robert A., RI]"


In [91]:
foo = pd.DataFrame.from_dict(info)
#foo['sponsor'] = info['sponsor']
#foo['cosponsors'] = info['cosponsors']
#foo['status'] = info['status']
#foo['top_subject'] = info['top_subject']
#foo['subjects'] = info['subjects']
foo.head(-1)

ValueError: arrays must all be same length

In [10]:
temp = pd.DataFrame.from_dict(info)

ValueError: arrays must all be same length

In [1]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from operator import attrgetter
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from tqdm import tqdm

#set up postgresql engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
cur = conn.cursor()

query = "SELECT * FROM allbills LIMIT 5;"
foo = pd.read_sql_query(query,engine)

Populating the interactive namespace from numpy and matplotlib


In [2]:
foo

,index,subjects,top_subject,bill_type,bill_number,status,result,sponsor,cosponsors,intro_date,final_date
0,0,"{""Administrative law and regulatory procedures...",Social welfare,s,1870,REPORTED,2,"{Sen,""Baucus, Max"",MT}",{},2013-12-19,2013-12-19
1,0,"{""Aquatic ecology"",""Congressional oversight"",F...",Water resources development,s,125,REFERRED,2,"{Sen,""Brown, Sherrod"",OH}","{{Sen,""Casey, Robert P., Jr."",PA},{Sen,""Franke...",2013-01-24,2013-01-24
2,0,"{""Congressional oversight"",""Department of Heal...",Health,s,1027,REFERRED,2,"{Sen,""Kirk, Mark Steven"",IL}","{{Sen,""Hatch, Orrin G."",UT},{Sen,""Johnson, Tim...",2013-05-22,2013-05-22
3,0,"{California,""Firearms and explosives"",Fires,""H...",Public lands and natural resources,s,1309,REPORTED,2,"{Sen,""Wyden, Ron"",OR}",{},2013-07-16,2014-05-14
4,0,"{Appropriations,Education,""Education programs ...",Education,s,2194,REFERRED,2,"{Sen,""Hirono, Mazie K."",HI}","{{Sen,""Gillibrand, Kirsten E."",NY},{Sen,""Marke...",2014-04-01,2014-04-01


In [5]:
print, foo.subjects.iloc[3]
bar = foo.subjects.iloc[3]
print(bar[1])

C
